In [57]:
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
from scipy.stats import multivariate_normal 
import matplotlib.pyplot as plt
from PIL import Image
import scipy.stats

In [241]:
class EM:
    def __init__(self, K, mus, sigmas, weights, tol_level, image_path):
        self.k = K # number of clusters
        self.mus = mus # means k x 1
        self.sigmas = sigmas # variances K x1
        self. weights = weights # wight of each cluster k x1
        self.tol_level = tol_level # the tolarance level
        self.data = np.array(Image.open(image_path).convert("L")).reshape(-1,1) #data points  n x 1
        # self.data = np.array([[1,1,2,3,4,3,5,7,1,2,3,4]])
        # self.data=  np.array(  [[1],
        #                         [1],
        #                         [2],
        #                         [3],
        #                         [4],
        #                         [3],
        #                         [5],
        #                         [7],
        #                         [1],
        #                         [2],
        #                         [3],
        #                         [4]])
        # print(self.data.shape)
        self.probabilities = np.array([0])# the probability for each class for each point N X K (2)
        

    def e_step(self): # prob of each data point for each cluster N X K and the new classes weights
        # we need the weights of the classes, parameters of the classes, data point
        
        temp = np.zeros( (self.data.shape[0], self.k)) # N X K
        for i in range(self.k):
            pd = scipy.stats.norm(self.mus[i],self.sigmas[i]).pdf(self.data)
            temp[:,i] = pd.reshape(-1)
        numerator = temp * self.weights.reshape(-1) # numerator done N X K (1)
        denominator = numerator.sum(axis=1)[:, np.newaxis]
        self.probabilities = numerator / denominator # the probability for each class for each point N X K (2)
        self.probabilities[np.isnan(self.probabilities)] = 0
        
        self.weights = self.probabilities.mean(axis=0).reshape(-1,1)
        
        
        

    def m_step(self):
        for i in range(self.k):
            weight = self.probabilities[:, [i]]
            total_weight = weight.sum()
            a1 = (self.data * weight).sum(axis=0)
            a2 = total_weight
            if a1 == 0 or a2 == 0:
                self.mus[i] = 0
            else:
                self.mus[i] = a1 / a2
        p1 = np.transpose(np.array([ self.data[:,0] for _ in range(0,self.k)]))
        p2 = np.array([ self.mus.reshape(-1) for _ in range(0,self.data.shape[0])]) # n x k
        p2 = np.square(p1 - p2)
        p2 = p2 * self.probabilities
        p2 = np.sum(p2,axis=0).reshape(-1,1)
        total_weight = np.sum(self.probabilities, axis=0).reshape(-1,1)
        p2 = p2 / total_weight
        p2[np.isnan(p2)] = 0
        self.sigmas = p2.reshape(-1,1)
        
    def q_objecive(self,old_mus, old_sigmas,old_weights):
        temp = np.zeros( (self.data.shape[0], self.k)) # N X K
        for i in range(self.k):
            pd = scipy.stats.norm(self.mus[i],self.sigmas[i]).pdf(self.data)
            temp[:,i] = pd.reshape(-1)
        temp1 = np.log(self.probabilities * temp)
        temp1[np.isnan(temp1)] = 0
        for i in range(self.k):
            pd = scipy.stats.norm(old_mus[i],old_sigmas[i]).pdf(self.data)
            temp[:,i] = pd.reshape(-1)
        
        numerator = temp * old_weights.reshape(-1)
        denominator = numerator.sum(axis=1)[:, np.newaxis]
        temp = numerator / denominator # the probability for each class for each point N X K (previous) 
        temp[np.isnan(temp)] = 0
        temp = temp * temp1
        temp = np.sum(temp, axis=1)
        temp = np.sum(temp, axis=0)
        if temp < self.tol_level:
            return True
        # print(temp)
        return False
        
    def cluster(self):
        stop = False
        self.e_step
        self.m_step
        itrs = 0
        while stop == False:
        # for i in range(0,1):
            itrs+=1
            old_mus = np.array(self.mus)
            old_sigs = np.array(self.sigmas)
            old_weights = np.array(self.weights)
            self.e_step()
            self.m_step()
            print(itrs)
            stop = self.q_objecive(old_mus, old_sigs, old_weights)
        return(self.mus, self.sigmas)
    
        


In [251]:
k = 4
phi = np.full(shape=k, fill_value=1/k) 
mu = np.array([[25],
               [90],
               [160],
               [240]])
sigma = np.array([[1],
               [1],
               [1],
               [1]])
e = 0.001

em1 = EM(k, mu, sigma, phi, e, r"dataset3\testGrayImage.jpg")
# em1.test()
results = em1.cluster()

1


In [252]:
print(results)

(array([[  0],
       [ 85],
       [170],
       [255]]), array([[0.],
       [0.],
       [0.],
       [0.]]))
